## After week 11:

In [16]:
import json
from pyspark.sql import Row
from pyspark.sql.functions import udf

In [17]:
@udf('string')
def munge_event(event_as_json):
    event = json.loads(event_as_json)
    event['Host'] = "moe"
    event['Cache-Control'] = "no-cache"
    return json.dumps(event)

In [18]:
raw_events = spark \
        .read \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "events") \
        .option("startingOffsets", "earliest") \
        .option("endingOffsets", "latest") \
        .load()

In [19]:
raw_events.show()

+----+--------------------+------+---------+------+--------------------+-------------+
| key|               value| topic|partition|offset|           timestamp|timestampType|
+----+--------------------+------+---------+------+--------------------+-------------+
|null|[7B 22 48 6F 73 7...|events|        0|     0|2020-12-07 21:44:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     1|2020-12-07 21:44:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     2|2020-12-07 21:44:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     3|2020-12-07 21:44:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     4|2020-12-07 21:44:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     5|2020-12-07 21:44:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     6|2020-12-07 21:44:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     7|2020-12-07 21:44:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0

In [20]:
munged_events = raw_events \
        .select(raw_events.value.cast('string').alias('raw'),
                raw_events.timestamp.cast('string')) \
        .withColumn('munged', munge_event('raw'))

In [21]:
munged_events.show()

+--------------------+--------------------+--------------------+
|                 raw|           timestamp|              munged|
+--------------------+--------------------+--------------------+
|{"Host": "user1.c...|2020-12-07 21:44:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-07 21:44:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-07 21:44:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-07 21:44:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-07 21:44:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-07 21:44:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-07 21:44:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-07 21:44:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-07 21:44:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-07 21:44:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-07 21:44:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-07 21:44:...|{"Host": "moe", "...|
|{"Host": "user1.c...|202

In [22]:
extracted_events = munged_events \
        .rdd \
        .map(lambda r: Row(timestamp=r.timestamp, **json.loads(r.munged))) \
        .toDF()

In [23]:
extracted_events.show()

Py4JJavaError: An error occurred while calling o220.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7, localhost, executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 6 fields are required while 7 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$.fromJava(EvaluatePython.scala:138)
	at org.apache.spark.sql.SparkSession$$anonfun$5.apply(SparkSession.scala:728)
	at org.apache.spark.sql.SparkSession$$anonfun$5.apply(SparkSession.scala:728)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:235)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:228)
	at org.apache.spark.rdd.RDD$$anonfun$mapPar

In [ ]:
sword_purchases = extracted_events \
        .filter(extracted_events.event_type == 'purchase_sword')
sword_purchases.show()

In [ ]:
default_hits = extracted_events \
        .filter(extracted_events.event_type == 'default')
default_hits.show()

## After week 12:

code from filtered_writes.py:

In [ ]:
import json
from pyspark.sql import Row
from pyspark.sql.functions import udf

In [ ]:
@udf('boolean')
def is_purchase(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'purchase_sword':
        return True
    return False

In [ ]:
raw_events = spark \
        .read \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "events") \
        .option("startingOffsets", "earliest") \
        .option("endingOffsets", "latest") \
        .load()

In [ ]:
purchase_events = raw_events \
        .select(raw_events.value.cast('string').alias('raw'),
                raw_events.timestamp.cast('string')) \
        .filter(is_purchase('raw'))

In [ ]:
extracted_purchase_events = purchase_events \
        .rdd \
        .map(lambda r: Row(timestamp=r.timestamp, **json.loads(r.raw))) \
        .toDF()

In [ ]:
extracted_purchase_events.printSchema()

In [ ]:
extracted_purchase_events.show()

In [ ]:
extracted_purchase_events \
        .write \
        .mode('overwrite') \
        .parquet('/tmp/purchases')

pyspark code

In [ ]:
purchases = spark.read.parquet('/tmp/purchases')

In [ ]:
purchases.show()

In [ ]:
purchases.registerTempTable('purchases')

In [ ]:
purchases_by_example2 = spark.sql("select * from purchases where Host = 'user1.comcast.com'")

In [ ]:
purchases_by_example2.show()

In [ ]:
df = purchases_by_example2.toPandas()
df.describe()

In [ ]:
df

### Business analytics questions:

simple analytics using spark sql on the spark dataframe in memory:

#### How many swords were purchased?

In [ ]:
swords = spark.sql("SELECT COUNT(*) FROM purchases").show()
swords

#### How many swords did comcast user 1 purchase?

In [ ]:
comcast_swords = spark.sql("select count(*) from purchases where Host = 'user1.comcast.com'").show()
comcast_swords

## After week 13:

In [ ]:
import json
from pyspark.sql.functions import udf, from_json
from pyspark.sql.types import StructType, StructField, StringType

In [ ]:
def purchase_sword_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
    ])

In [ ]:
@udf('boolean')
def is_sword_purchase(event_as_json):
    """udf for filtering events
    """
    event = json.loads(event_as_json)
    if event['event_type'] == 'purchase_sword':
        return True
    return False

In [24]:
raw_events = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "events") \
        .load()

In [25]:
sword_purchases = raw_events \
        .filter(is_sword_purchase(raw_events.value.cast('string'))) \
        .select(raw_events.value.cast('string').alias('raw_event'),
                raw_events.timestamp.cast('string'),
                from_json(raw_events.value.cast('string'),
                          purchase_sword_event_schema()).alias('json')) \
        .select('raw_event', 'timestamp', 'json.*')

In [26]:
sink = sword_purchases \
        .writeStream \
        .format("parquet") \
        .option("checkpointLocation", "/tmp/checkpoints_for_sword_purchases") \
        .option("path", "/tmp/sword_purchases") \
        .trigger(processingTime="10 seconds") \
        .start()

Py4JJavaError: An error occurred while calling o299.start.
: java.lang.IllegalStateException: Cannot start query with id 2551cf16-94be-474c-8806-d7cd1401ff10 as another query with same id is already active. Perhaps you are attempting to restart a query from checkpoint that is already active.
	at org.apache.spark.sql.streaming.StreamingQueryManager.startQuery(StreamingQueryManager.scala:300)
	at org.apache.spark.sql.streaming.DataStreamWriter.start(DataStreamWriter.scala:282)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:280)
	at py4j.commands.AbstractCommand.invokeMethod(Abstra

In [27]:
sink.stop()

#### Include your hive code to create schema on read:

docker-compose exec cloudera hive

create external table if not exists default.sword_purchases (Accept string, Host string, User_Agent string, event_type string, timestamp string) stored as parquet location '/tmp/sword_purchases'  tblproperties ("parquet.compress"="SNAPPY");

#### Include the query and the first few lines of the result or use a limit 5, etc:

docker-compose exec presto presto --server presto:8080 --catalog hive --schema default

QUERIES:
select * from sword_purchases;
- RESULT:
{"Host": "user1.comcast.com", "sword_type": "knights", "event_type": "purchase_sword", "Accept": "*/*", "User-A
 {"Host": "user1.comcast.com", "sword_type": "knights", "event_type": "purchase_sword", "Accept": "*/*", "User-A
 {"Host": "user1.comcast.com", "sword_type": "knights", "event_type": "purchase_sword", "Accept": "*/*", "User-A
 {"Host": "user1.comcast.com", "sword_type": "knights", "event_type": "purchase_sword", "Accept": "*/*", "User-A
 {"Host": "user1.comcast.com", "sword_type": "knights", "event_type": "purchase_sword", "Accept": "*/*", "User-A
 
 
select count(*) from sword_purchases;
- RESULT:
_col0 
-------
  1750 
(1 row)

#### Spin up the cluster:
- docker-compose up -d
- docker-compose ps
- docker ps -a

#### Tear down the cluster:
- docker-compose down
- docker-compose ps
- docker ps -a

## APPENDIX: DESCRIPTION OF COMMAND LINE CODE IN LINUX, WEEK BY WEEK

## Week 11:

Copy in the yml file:
cp ~/w205/course-content/11-Storing-Data-III/docker-compose.yml ~/w205/project-3-kt-cason

Spin up the cluster:
docker-compose up -d
docker-compose ps
docker ps -a

Copy in the game.api:
cp ~/w205/course-content/11-Storing-Data-III/game_api.py ~/w205/project-3-kt-cason

Create a topic:
docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

Start flask:
docker-compose exec mids env FLASK_APP=/w205/project-3-kt-cason/game_api.py flask run --host 0.0.0.0

Generate events using curl:
docker-compose exec mids curl http://localhost:5000/
docker-compose exec mids curl http://localhost:5000/purchase_a_sword
docker-compose exec mids curl http://localhost:5000/buy_a_sword
docker-compose exec mids curl http://localhost:5000/join_guild

Read from Kafka:
docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning -e

Create a Jupyter notebook:
docker-compose exec spark bash
ln -s /w205 w205
Exit
	docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark

http://34.82.31.218:8888/?token=c6264b9f9d99e8f34893d649324c5e322fe3ffda54bfc9ec

Tear down cluster:
docker-compose down
docker-compose ps
docker ps -a

## Week 12:

WEEK 12:
Copy in the yml file:
cp ~/w205/course-content/12-Querying-Data-II/docker-compose.yml ~/w205/project-3-kt-cason

Spin up the cluster:
docker-compose up -d
Docker-compose ps
Docker ps -a

Modify in the game.api:
Add meta data vi game_api

Create a topic:
docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

Start flask:
docker-compose exec mids env FLASK_APP=/w205/project-3-kt-cason/game_api.py flask run --host 0.0.0.0

Apache bench to generate data:
USER 1 at Comcast:

docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/
docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/purchase_a_sword
docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/buy_a_sword
docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/join_guild

USER 2 at ATT:
docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/
docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/purchase_a_sword
docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/buy_a_sword
docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/join_guild

Verify there all on there:
docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning -e

Create a Jupyter notebook:
docker-compose exec spark bash
ln -s /w205 w205
Exit
	docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark

 http://34.82.31.218:8888/?token=cdb798175674286bae4a6236c4895e848b8e0e4cbd90d5eb

Tear down cluster:
docker-compose down
Docker-compose ps
Docker ps -a

## Week 13:

Copy in the yml file:
cp ~/w205/course-content/13-Understanding-Data/docker-compose.yml ~/w205/project-3-kt-cason

vi docker-compose.yml: edit yml file
Comment out ports in Cloudera
Remove hash on ports in spark

Spin up the cluster:
docker-compose up -d
Docker-compose ps
Docker ps -a

Start game api:
cp ~/w205/course-content/11-Storing-Data-III/game_api.py ~/w205/project-3-kt-cason

Create a topic:
docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

Start flask:
docker-compose exec mids env FLASK_APP=/w205/project-3-kt-cason/game_api.py flask run --host 0.0.0.0

Infinite loop:
while true; do docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/purchase_a_sword; sleep 5; done

Verify there all on there:
docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning -e

Create a Jupyter notebook:
docker-compose exec spark bash
ln -s /w205 w205
Exit
	docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark

http://34.82.31.218:8888/?token=4d836d5a16b391112cbf7da44bebc7f8a11212bb2791ed18

Create hive:
docker-compose exec cloudera hive

create external table if not exists default.sword_purchases (Accept string, Host string, User_Agent string, event_type string, timestamp string) stored as parquet location '/tmp/sword_purchases'  tblproperties ("parquet.compress"="SNAPPY");

Code to start presto:
docker-compose exec presto presto --server presto:8080 --catalog hive --schema default

Run some queries:
Select * from sword_purchases;
Select count(*) from sword_purchases;

Tear down cluster:
docker-compose down
Docker-compose ps
Docker ps -a